## Face detection + confidence score 

In [2]:
# !pip install mediapipe

import cv2
import mediapipe as mp

print(cv2.__version__)
print(mp.__version__)


4.11.0
0.10.21


In [10]:
import mediapipe as mp
from mediapipe import solutions
import numpy as np
import cv2


# Initialize Mediapipe face mesh and face detection
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection

face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_mesh_results = face_mesh.process(rgb_frame)
    face_detection_results = face_detection.process(rgb_frame)

    if face_detection_results.detections:
        # If face is detected, get the detection confidence
        confidence = face_detection_results.detections[0].score[0]

        # Draw the detection box around the face
        for detection in face_detection_results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display detection confidence
        cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    if face_mesh_results.multi_face_landmarks:
        for face_landmarks in face_mesh_results.multi_face_landmarks:
            # Get the number of landmarks detected
            num_landmarks = len(face_landmarks.landmark)

            # Draw the landmarks
            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

            # Display the number of landmarks detected
            cv2.putText(frame, f"Landmarks: {num_landmarks}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame
    cv2.imshow("Real-Time Face Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

#### Eye tracker + blinking 

In [11]:
import mediapipe as mp
import cv2
import numpy as np

# Initialize Mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Eye landmark indices (left and right eye)
LEFT_EYE_LANDMARKS = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_LANDMARKS = [362, 385, 387, 263, 373, 380]

# Function to calculate EAR
def calculate_ear(landmarks, eye_points):
    """Compute the Eye Aspect Ratio (EAR) to determine if an eye is open or closed."""
    p1 = np.array([landmarks[eye_points[1]].x, landmarks[eye_points[1]].y])
    p2 = np.array([landmarks[eye_points[2]].x, landmarks[eye_points[2]].y])
    p3 = np.array([landmarks[eye_points[4]].x, landmarks[eye_points[4]].y])
    p4 = np.array([landmarks[eye_points[5]].x, landmarks[eye_points[5]].y])
    p5 = np.array([landmarks[eye_points[0]].x, landmarks[eye_points[0]].y])
    p6 = np.array([landmarks[eye_points[3]].x, landmarks[eye_points[3]].y])

    # Vertical distances
    vertical_1 = np.linalg.norm(p2 - p4)
    vertical_2 = np.linalg.norm(p3 - p5)
    
    # Horizontal distance
    horizontal = np.linalg.norm(p1 - p6)
    
    # EAR formula
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Threshold for blink detection
EAR_THRESHOLD = 0.81
BLINK_FRAMES = 1 # Number of consecutive frames to count as a blink

blink_count = 0
frame_counter = 0
blinked = False

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Compute EAR for both eyes
            left_ear = calculate_ear(face_landmarks.landmark, LEFT_EYE_LANDMARKS)
            right_ear = calculate_ear(face_landmarks.landmark, RIGHT_EYE_LANDMARKS)
            avg_ear = (left_ear + right_ear) / 2.0

            # Detect blink
            if avg_ear < EAR_THRESHOLD:
                frame_counter += 1
            else:
                if frame_counter >= BLINK_FRAMES:
                    blink_count += 1
                    blinked = True
                frame_counter = 0

            # Display EAR and Blink Count
            cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, f"Blinks: {blink_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow("Blink Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


### Drowsiness Detection from here

In [3]:
pip install tensorflow opencv-python mediapipe numpy matplotlib


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
     ------------------------ --------------- 30.7/50.1 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 50.1/50.1 kB 847.4 kB/s eta 0:00:00
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cach


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import shutil
import random

# Define dataset paths
source_dir = "This is the path/Driver Drowsiness Dataset (DDD)"
target_dir = "path to /Processed_Dataset"

# Classes
classes = ["Drowsy", "Non Drowsy"]

# Create target directories
os.makedirs(target_dir, exist_ok=True)
for c in classes:
    os.makedirs(os.path.join(target_dir, c), exist_ok=True)

# Sample 1000 images per class
num_samples = 10000

for c in classes:
    source_path = os.path.join(source_dir, c)
    target_path = os.path.join(target_dir, c)

    # Get all images
    images = os.listdir(source_path)

    # Randomly select 1000 images
    selected_images = random.sample(images, num_samples)

    # Copy images to new dataset
    for img in selected_images:
        shutil.copy(os.path.join(source_path, img), os.path.join(target_path, img))

print("✅ Dataset prepared with 10000 images per class!")


✅ Dataset prepared with 10000 images per class!


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define dataset paths
train_dir = "path to /Processed_Dataset"

# Image Data Generator (Preprocessing)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load train and validation sets
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 16806 images belonging to 2 classes.
Found 4201 images belonging to 2 classes.


In [6]:
# Build CNN Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Regularization
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary Classification
])

# Compile Model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save the model
model.save("drowsiness_model.h5")
print("✅ Model training complete and saved as drowsiness_model.h5")


Epoch 1/10
 13/526 ━━━━━━━━━━━━━━━━━━━━ 2:55 342ms/step - accuracy: 0.4767 - loss: 1.0237

KeyboardInterrupt: 

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("drowsiness_model.h5")

# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_detection_results = face_detection.process(rgb_frame)

    if face_detection_results.detections:
        for detection in face_detection_results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            # Crop the detected face
            face = frame[y:y+h, x:x+w]

            # Preprocess for model
            face_resized = cv2.resize(face, (224, 224))
            face_resized = face_resized / 255.0
            face_resized = np.expand_dims(face_resized, axis=0)

            # Predict drowsiness
            prediction = model.predict(face_resized)[0][0]
            status = "Drowsy" if prediction > 0.5 else "Alert"

            # Draw detection box & status
            color = (0, 0, 255) if status == "Drowsy" else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, status, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Show frame
    cv2.imshow("Driver Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━

## Training on personal generated dataset

In [ ]:
import os

# Define dataset directory
dataset_dir = "path to/Custom_Dataset"
drowsy_dir = os.path.join(dataset_dir, "Drowsy")
non_drowsy_dir = os.path.join(dataset_dir, "Non_Drowsy")

# Create directories if they don't exist
os.makedirs(drowsy_dir, exist_ok=True)
os.makedirs(non_drowsy_dir, exist_ok=True)

print("✅ Dataset folders created!")


✅ Dataset folders created!


## Non_Drowsy images saved to the Non_Drowsy folder

In [ ]:
import cv2

# Define dataset paths
# dataset_dir = "C:/Repos/Smartroads/SmartRoads-AI/dev/Custom_Dataset"
dataset_dir = "path to/Custom_Dataset"
# Open webcam
cap = cv2.VideoCapture(0)
counter = 0
category = "Non_Drowsy"  # Change to "Drowsy" when needed

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Show the frame
    cv2.imshow(f"Capture {category} Images - Press 's' to Save, 'q' to Quit", frame)

    # Capture image when 's' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord("s"):
        img_path = os.path.join(dataset_dir, category, f"{category}_{counter}.jpg")
        cv2.imwrite(img_path, frame)
        counter += 1
        print(f"✅ Saved: {img_path}")

    # Quit when 'q' is pressed
    elif key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


✅ Saved: C:/Repos/Smartroads/SmartRoads-AI/dev/Sprint2_face_detection_model/Custom_Dataset\Non_Drowsy\Non_Drowsy_0.jpg
✅ Saved: C:/Repos/Smartroads/SmartRoads-AI/dev/Sprint2_face_detection_model/Custom_Dataset\Non_Drowsy\Non_Drowsy_1.jpg
✅ Saved: C:/Repos/Smartroads/SmartRoads-AI/dev/Sprint2_face_detection_model/Custom_Dataset\Non_Drowsy\Non_Drowsy_2.jpg
✅ Saved: C:/Repos/Smartroads/SmartRoads-AI/dev/Sprint2_face_detection_model/Custom_Dataset\Non_Drowsy\Non_Drowsy_3.jpg
✅ Saved: C:/Repos/Smartroads/SmartRoads-AI/dev/Sprint2_face_detection_model/Custom_Dataset\Non_Drowsy\Non_Drowsy_4.jpg
